In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

2024-05-09 13:44:59.084555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# m = 2000
# X, y = make_blobs(m, centers=([2.5, 3], [6.5, 8], [2, 8], [7.5, 3]), random_state=0)
# y[(y == 0) | (y == 1)] = 1
# y[(y == 2) | (y == 3)] = 0

# y = np.reshape(y, (-1, 1))

# plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='white', cmap='Dark2', linewidths=0.2);

# print ('The shape of X is: ' + str(X.shape))
# print ('The shape of Y is: ' + str(y.shape))

In [3]:
# X = pd.DataFrame(X)

In [4]:
df = pd.read_csv('house_prices_train.csv', index_col='Id', na_filter=True)

In [5]:
df = pd.get_dummies(df, drop_first=True)
df.corrwith(df['SalePrice']).sort_values()

ExterQual_TA        -0.589044
KitchenQual_TA      -0.519298
BsmtQual_TA         -0.452394
GarageFinish_Unf    -0.410608
GarageType_Detchd   -0.354141
                       ...   
GarageArea           0.623431
GarageCars           0.640409
GrLivArea            0.708624
OverallQual          0.790982
SalePrice            1.000000
Length: 245, dtype: float64

In [6]:
X = df[['OverallQual', 'GrLivArea']]
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_norm = (X_train - X.mean()) / X.std()
X_test_norm = (X_test - X.mean()) / X.std()
y_train_norm = (y_train - y.mean()) / y.std()

y_train_norm = np.reshape(y_train_norm, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

In [7]:
def initialize_parameters(n_x, n_h, n_y):
    
    W1 = tf.Variable(tf.random.normal((n_x, n_h)), name='W1')
    b1 = tf.Variable(tf.random.normal((1, n_h)), name='b1')
    W2 = tf.Variable(tf.random.normal((n_h, n_y)), name='W2')
    b2 = tf.Variable(tf.random.normal((1, n_y)), name='b2')

    return W1, b1, W2, b2

In [8]:
# def sigmoid(z):
#     return 1/(1+tf.exp(-z))

In [9]:
# def forward_propagation(X, W1, b1, W2, b2):
#     Z1 = X.to_numpy() @ W1 + b1
#     A1 = sigmoid(Z1) 
#     Z2 = A1 @ W2 + b2
#     y_hat = sigmoid(Z2)
#     return y_hat 

In [10]:
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = X.to_numpy() @ W1 + b1
    # A1 = sigmoid(Z1) 
    y_hat = Z1 @ W2 + b2
    # y_hat = sigmoid(Z2)
    return y_hat 

In [11]:
# def compute_cost(y, y_hat):
#     m = y.shape[0]
#     logloss = - np.multiply(np.log(y_hat), y) - np.multiply(np.log(1 - y_hat), 1 - y)
#     cost = 1/m * tf.reduce_sum(logloss)
#     return cost

In [12]:
def compute_cost(y, y_hat):
    m = y.shape[0]
    cost = tf.reduce_sum((y - y_hat) ** 2) / (2 * m)
    return cost

In [13]:
def gradiant_descent(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):

    W1.assign_sub(learning_rate * dW1)
    b1.assign_sub(learning_rate * db1)
    W2.assign_sub(learning_rate * dW2)
    b2.assign_sub(learning_rate * db2)

    return W1, b1, W2, b2


In [14]:
def nn_model(X, y, n_x, n_h, n_y, epochs, learning_rate):
    W1, b1, W2, b2 = initialize_parameters(n_x, n_h, n_y)
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            y_hat = forward_propagation(X, W1, b1, W2, b2)
            cost = compute_cost(y, y_hat)
            
            if epoch % 10 == 0:
                print(f'Epoch:{epoch}, Cost: {cost}')
        
        # Backward Propagation
        dW1, db1, dW2, db2 = tape.gradient(cost, [W1, b1, W2, b2])
        
        W1, b1, W2, b2 = gradiant_descent(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

    return W1.numpy(), b1.numpy(), W2.numpy(), b2.numpy()

In [15]:
LEARNING_RATE = 0.05
EPOCHS = 200
n_x = X.shape[1]    # number of inputs
n_h = 2             # number of perceptrons in hidden layer
n_y = 1    # number of perceptrons in output layer

W1, b1, W2, b2 = nn_model(X_train_norm, y_train_norm, n_x, n_h, n_y, EPOCHS, LEARNING_RATE)

2024-05-09 13:45:03.170343: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch:0, Cost: 4.224335670471191
Epoch:10, Cost: 0.1986522525548935
Epoch:20, Cost: 0.14519010484218597
Epoch:30, Cost: 0.14151735603809357
Epoch:40, Cost: 0.14126431941986084
Epoch:50, Cost: 0.14124691486358643
Epoch:60, Cost: 0.1412457376718521
Epoch:70, Cost: 0.14124564826488495
Epoch:80, Cost: 0.14124566316604614
Epoch:90, Cost: 0.14124564826488495
Epoch:100, Cost: 0.14124564826488495
Epoch:110, Cost: 0.14124563336372375
Epoch:120, Cost: 0.14124563336372375
Epoch:130, Cost: 0.14124563336372375
Epoch:140, Cost: 0.14124563336372375
Epoch:150, Cost: 0.14124563336372375
Epoch:160, Cost: 0.14124563336372375
Epoch:170, Cost: 0.14124563336372375
Epoch:180, Cost: 0.14124563336372375
Epoch:190, Cost: 0.14124563336372375
